In [2]:
import re
import time
import pandas as pd
import numpy as np
import nltk
from nltk import *
import json

import random
random.seed(10)

from joblib import dump, load

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score

import tensorflow as tf


# Ejemplo con Stack de RNNs
from tensorflow.keras.layers import Embedding, SimpleRNN
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, LSTM, SpatialDropout1D
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.losses import SparseCategoricalCrossentropy

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score #son metricas - area debajo de la curva roc

from sklearn.model_selection import train_test_split




### Lectura del archivo de tweets ingestados y etiquetados

In [3]:
corpus_train_esA = pd.read_csv('tweets_search_etiquetas.csv',delimiter='\t',encoding='latin-1', dtype = {'id_str': str})
corpus_train_esA["id_str"] = corpus_train_esA["id_str"].astype(str).replace(to_replace="[.]", value = "", regex=True)
corpus_train_esA["id_str"] = corpus_train_esA["id_str"].astype(str).replace(to_replace="E\+\d{3}", value = "", regex=True)

corpus_train_esA.head(10)

,id_str,created_at,screen_name,text,user_followers,sentiment,funcionality,client_attention
0,153653000000000,Tue Jun 14 01:55:56 +0000 2022,EdgarcueroO,¡Disfruta conmigo de llamadas gratis en @Dingt...,5006,0,0,0
1,153653000000000,Tue Jun 14 01:51:30 +0000 2022,gilcuate,RT @IvanNavaMx: Ayudanos a vender 10 boletos d...,20,0,0,0
2,153653000000000,Tue Jun 14 01:45:56 +0000 2022,ilovminnie_,me cambie a Huawei y ahora no se cómo entrar a...,51,0,1,0
3,153653000000000,Tue Jun 14 01:14:19 +0000 2022,alexander_tdf,RT @Lauriita18_: NECESITO VUESTRA AYUDA: Que ...,6,0,0,0
4,153653000000000,Tue Jun 14 01:06:54 +0000 2022,pablo_calva,@jere_cob24 @HuaweiMobileMx Yo no :( aún sigo ...,39,0,1,0
5,153653000000000,Tue Jun 14 00:30:12 +0000 2022,Zavala_Ra,Huawei podría aportar a la educación superior ...,7006,0,0,0
6,153653000000000,Tue Jun 14 00:28:08 +0000 2022,coosti_b,será este el fin del huawei GW?,250,0,1,0
7,153653000000000,Mon Jun 13 23:53:43 +0000 2022,eveemtz_,"@MAULvc7 Me resistiré lo mas que pueda, hasta ...",34,0,0,0
8,153653000000000,Mon Jun 13 23:46:33 +0000 2022,blogyarq,"@MAULvc7 Es que amo Huawei, tengo el p30 lite ...",70643,1,1,0
9,153653000000000,Mon Jun 13 23:44:29 +0000 2022,blogydeco,En estos momentos odio a Huawei??,14568,0,1,0


### Limpieza en los datos
* Cambiar todas las palabras de mayúsculas a minúsculas
* Se han eliminado las '@' de @USUARIO con el fin de facilitar el etiquetado morfológico
* Quitar los links 
* Quitar los emojis
* Eliminar las stopwords
* Se han reemplazado todos los números por el símbolo '0'
* Quitar los signos de puntuación y quitar espacios (tabuladores, etc)


In [4]:
pattern_URL="(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9]\.[^\s]{2,})"

def procesar(file, namefile):    
    file[file.columns[3]] = [clean_text(i) for i in file[file.columns[3]]]    

    sentiment_code_dict = {"000":0, "010":1, "001":2, "110":3, "101":4, "011": 5} 
    file["sentiment_code"] = file["sentiment"].map(str)  + file["funcionality"].map(str)  + file["client_attention"].map(str) 
    file["sentiment_code"] = file["sentiment_code"].apply(lambda x: sentiment_code_dict[x])

    file.to_csv(namefile, sep=';', encoding='latin-1', index=False)
    return file
    
def clean_text(text):
    text = text.lower()   
    #text=re.sub("@([A-Za-z0-9_]{1,15})", "@USUARIO", text)
    text=re.sub("@([A-Za-z0-9_]{1,15})", "", text)
    text=re.sub(pattern_URL, "", text)
    
    text= remove_emoji(text)
    text = re.sub("(\d+)|(rt)|(RT)", "", text)
    text = re.sub("#\w+", "", text)
    text= remove_stopwords(text)
    
    # text=re.sub("\d+", " ", text)
    
    text=re.sub(r" +", " ", re.sub(r"\t", " ", re.sub(r"\n+", "\n", re.sub('(?:[.,\/!$%?¿?!¡\^&\*;:{}=><\-_`~()”“"\'\|])', " ",text))))
    text = text.strip()
    return text

def remove_stopwords(text):    
    nltk.download('stopwords')
    stopwords=set(nltk.corpus.stopwords.words("spanish"))
    for i in stopwords:
        text = re.sub(r"\b%s\b" % i, " ", text)
    return text

def remove_emoji(text):
    emoji_pattern = re.compile("["
                               "\U0001F600-\U0001F64F"  # emoticons
                               "\U0001F300-\U0001F5FF"  # symbols & pictographs                               
                               "\U0001F680-\U0001F6FF"  # transport & map symbols
                               "\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "\U00002702-\U000027B0"
                               "\U000024C2-\U0001F251"
                               "\U0001f926-\U0001f937"
                               "\u200d"
                               "\u2640-\u2642"
                               "\U0001F1F2-\U0001F1F4"  # Macau flag
                               "\U0001F1E6-\U0001F1FF"  # flags
                               "\U0001F600-\U0001F64F"
                               "\U0001F1F2"
                               "\U0001F1F4"
                               "\U0001F620"
                               "]+", flags=re.UNICODE)   
    text = emoji_pattern.sub(r'', text) # no emoji

    return text

In [5]:
corpus_train_esA = procesar(corpus_train_esA, "tweets_search_etiquetas_clean.csv")
corpus_train_esA.tail()
#corpus_train_esA = pd.read_csv("tweets_search_etiquetas_clean.csv", sep = ";", encoding = "latin-1")
#corpus_train_esA.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\histe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\histe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\histe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\histe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\histe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\histe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopword

,id_str,created_at,screen_name,text,user_followers,sentiment,funcionality,client_attention,sentiment_code
311,153653000000000,Tue Jun 20 01:55:56 +0000 2022,pruebaTrainDummy,colores fotos huawei estan saturados malos,1,0,1,0,1
312,153653000000000,Tue Jun 20 01:55:56 +0000 2022,pruebaTrainDummy,contento huawei colores estan saturados,1,0,1,0,1
313,155237976078913,Wed Jul 27 19:45:51 +0000 2022,unamushasha,si huawei varian datos ejemplo dice dormi xd,2088,0,1,1,5
314,15523789805773,Wed Jul 27 19:42:45 +0000 2022,Andre_Villalba,buen teléfono huawei android malo so propietar...,358,1,1,0,3
315,155237713161064,Wed Jul 27 19:35:24 +0000 2022,loboalonsotegui,llevo huawei pesar fallos mu tontos ahí sigue ...,132,1,1,0,3


### Particionamiento de los datos en conjunto de entrenamiento y conjunto de pruebas

In [6]:
corpus_train_esA, corpus_dev_esA = train_test_split(corpus_train_esA, test_size=0.3)

corpus_dev_esA.to_csv("tweets_search_etiquetas_clean_test.csv", sep = ";", encoding = "latin-1", index = False)
corpus_dev_esA = pd.read_csv("tweets_search_etiquetas_clean_test.csv", sep = ";", encoding = "latin-1")
corpus_dev_esA.tail()

,id_str,created_at,screen_name,text,user_followers,sentiment,funcionality,client_attention,sentiment_code
90,153681198832356,Tue Jun 14 20:45:05 +0000 2022,_hidalgovv,sacaba lindas fotos luna huawei,64,1,1,0,3
91,153653000000000,Mon Jun 13 13:17:35 +0000 2022,Atilaa27,huawei patenta procesador cuántico problemas a...,1916,0,0,0,0
92,153653000000000,Tue Jun 20 01:55:56 +0000 2022,pruebaTrainDummy,camara huawei modo retrato basura recoa siluet...,1,0,1,0,1
93,153677391512049,Tue Jun 14 18:13:47 +0000 2022,Revolucion_Web,obtén velocidad transmisión tarjetas capacidad...,59,0,0,0,0
94,153667758493469,Tue Jun 14 11:51:00 +0000 2022,PDD20,hola cualquier huawei lanzado mayo servicios g...,3394,0,1,0,1


In [7]:
train_idA = corpus_train_esA[corpus_train_esA.columns[0]]
X_train_textA = corpus_train_esA[corpus_train_esA.columns[3]].fillna(' ')
#y_train_hsA = corpus_train_esA[[corpus_train_esA.columns[5], corpus_train_esA.columns[6], corpus_train_esA.columns[7]]]
y_train_hsA = corpus_train_esA[corpus_train_esA.columns[5]]
y_train_hsA = y_train_hsA.values

test_idA = corpus_dev_esA[corpus_train_esA.columns[0]]
X_test_textA = corpus_dev_esA[corpus_dev_esA.columns[3]].fillna(' ')
#y_test_hsA = corpus_dev_esA[[corpus_dev_esA.columns[5], corpus_dev_esA.columns[6], corpus_dev_esA.columns[7]]]
y_test_hsA = corpus_dev_esA[corpus_dev_esA.columns[5]]
y_test_hsA = y_test_hsA.values

In [8]:
print(y_train_hsA.shape)

(221,)


In [9]:
 # CountVectorizer creates a vector with lenght equal to the n of unique words in corupus. 1 indicates
# that the word is present
cvectorizer = CountVectorizer(
    # lowercase=True,
    #stop_words=[word.decode('utf-8') for word in nltk.corpus.stopwords.words('spanish')],
    #token_pattern=r'\b\w+\b', #selects tokens of 2 or more alphanumeric characters 
    ngram_range = (3,5),#n-grams de palabras n = 1 a n = 3 (unigramas, bigramas y trigramas)
    min_df = 3,#ignorando los términos que tienen una frecuencia de documento estrictamente inferior a 5
).fit(X_train_textA) # Identifica las palabras unicas y las coloca en un vector

X_train_cvectorized = cvectorizer.transform(X_train_textA).toarray() # para cada palabra unica del corpus, coloca 1 si la palabra aparece en el texto
print(X_train_cvectorized.shape)

X_test_cvectorized = cvectorizer.transform(X_test_textA).toarray()
print(X_test_cvectorized.shape)

dump(cvectorizer, 'count_vectorizer.joblib') # Guarda el vectorizer


(221, 63)
(95, 63)


['count_vectorizer.joblib']

In [10]:
print(len(X_train_cvectorized[0]))
print(X_test_cvectorized.shape)

63
(95, 63)


In [11]:
#X_train_cvectorized
y_train_hsA

array([0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       0], dtype=int64)

In [12]:
cvectorizer.vocabulary_

{'van dos semanas': 60,
 'dos semanas sigo': 16,
 'semanas sigo esperando': 50,
 'sigo esperando respuesta': 53,
 'esperando respuesta huawei': 20,
 'respuesta huawei prometen': 47,
 'huawei prometen cupones': 31,
 'prometen cupones pides': 44,
 'cupones pides ignoran': 12,
 'pides ignoran teléfono': 40,
 'van dos semanas sigo': 61,
 'dos semanas sigo esperando': 17,
 'semanas sigo esperando respuesta': 51,
 'sigo esperando respuesta huawei': 54,
 'esperando respuesta huawei prometen': 21,
 'respuesta huawei prometen cupones': 48,
 'huawei prometen cupones pides': 32,
 'prometen cupones pides ignoran': 45,
 'cupones pides ignoran teléfono': 13,
 'van dos semanas sigo esperando': 62,
 'dos semanas sigo esperando respuesta': 18,
 'semanas sigo esperando respuesta huawei': 52,
 'sigo esperando respuesta huawei prometen': 55,
 'esperando respuesta huawei prometen cupones': 22,
 'respuesta huawei prometen cupones pides': 49,
 'huawei prometen cupones pides ignoran': 33,
 'prometen cupones p

In [13]:
print(X_test_cvectorized)
print(X_test_cvectorized.shape)
print(len(cvectorizer.vocabulary_))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(95, 63)
63


In [14]:
# Save the vocabulary in json format
with open("cvectorizer_vocabulary_.json", mode = "w", encoding = "latin-1", ) as f:
    f.write(json.dumps(cvectorizer.vocabulary_, indent = 4, ensure_ascii = False))


### Network Model

In [15]:
max_features =  len(cvectorizer.vocabulary_)# or X_train_cvectorized.shape[1] #10000  # tamaño del diccionario de palabras comunes
                      # (número de palabras a utilizar)
maxlen = X_test_cvectorized.shape[1]  # 67         # longitud máxima de cada secuencia de entrenamiento
batch_size = 32

In [16]:
X_train_cvectorized

array([[0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [ ]:
# categorical_crossentropy (cce) produces a one-hot array in the output containing the probable match for each category
# when you have only one class per record. This uses softmax, due to this is a multi class problem [0, 1, 0].
# sparse_categorical_crossentropy (scce) The vector isn't a one hot, because per record you have one element. [2], pred [.2,.3,.4]
# and then this produce a category index of the most likely matching category.
# binary_crossentropy: There you have more than one class per record and more than one class. [1,1,0] or simple binary
# this loss function uses sigmod, because is a bniary problem

model = Sequential()
# Capa embedding. Convierte los tokens en vectores densos de dimension fija
# input_dim : size of vocavulary (Cada palabra o token se convirtirá en un vector)
# output_dim: dimensión del vector al que se mapea
# input_length. is the length of tthe tokens by record 
# output_dim is the size of the embedding vector for each word in the vocabulary.
#model.add(Embedding(input_dim = max_features,  output_dim=32, input_length = 15)) # output_dim = 32
model.add(Dense(10, activation = "relu", input_shape = (len(cvectorizer.vocabulary_),) ))
#model.add(SimpleRNN(32, return_sequences=True)) # van despues de embeding
#model.add(SimpleRNN(32))

model.add(Dense(1, activation='sigmoid')) # softmax para multicalse. sigmoid to binary classification

model.summary()

#model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

model.compile(optimizer='adam', loss = "categorical_crossentropy", metrics=['accuracy'])

c:\Users\histe\Documents\formacion\data_science_IA\proyecto_twitter_tech_sentiment_analysis\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_11 (Dense)                │ (None, 10)             │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 651 (2.54 KB)

 Trainable params: 651 (2.54 KB)

 Non-trainable params: 0 (0.00 B)

In [37]:

X_train_cvectorized.shape[1]

63

In [1]:
y_train_hsA

NameError: name 'y_train_hsA' is not defined

In [41]:

tic = time.time()


history_stackRNN = model.fit(
    X_train_cvectorized, y_train_hsA,
    epochs=15,
    batch_size=batch_size,
    validation_split=0.2,
    verbose=2
)
print('Tiempo de entrenamiento:', time.time()-tic)

# evaluate the model
scores = model.evaluate(X_test_cvectorized, y_test_hsA, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[0], scores[0]*100))

loss, accuracy = model.evaluate(X_train_cvectorized, y_train_hsA, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy)) # accuracy. Exactitud. total de Positives y Falses que son True sobre el total de todo
loss, accuracy = model.evaluate(X_test_cvectorized, y_test_hsA, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
#plot_history(history)

# make predictions
testPredict_stackRNN = model.predict(X_test_cvectorized)
print('\t', 'Accuracy', accuracy_score(y_test_hsA, testPredict_stackRNN.round()))

Epoch 1/15
6/6 - 0s - 15ms/step - accuracy: 0.6875 - loss: 0.0000e+00 - val_accuracy: 0.7111 - val_loss: 0.0000e+00
Epoch 2/15
6/6 - 0s - 12ms/step - accuracy: 0.6875 - loss: 0.0000e+00 - val_accuracy: 0.7111 - val_loss: 0.0000e+00
Epoch 3/15
6/6 - 0s - 12ms/step - accuracy: 0.6875 - loss: 0.0000e+00 - val_accuracy: 0.7111 - val_loss: 0.0000e+00
Epoch 4/15
6/6 - 0s - 12ms/step - accuracy: 0.6875 - loss: 0.0000e+00 - val_accuracy: 0.7111 - val_loss: 0.0000e+00
Epoch 5/15
6/6 - 0s - 11ms/step - accuracy: 0.6875 - loss: 0.0000e+00 - val_accuracy: 0.7111 - val_loss: 0.0000e+00
Epoch 6/15
6/6 - 0s - 11ms/step - accuracy: 0.6875 - loss: 0.0000e+00 - val_accuracy: 0.7111 - val_loss: 0.0000e+00
Epoch 7/15
6/6 - 0s - 12ms/step - accuracy: 0.6875 - loss: 0.0000e+00 - val_accuracy: 0.7111 - val_loss: 0.0000e+00
Epoch 8/15
6/6 - 0s - 11ms/step - accuracy: 0.6875 - loss: 0.0000e+00 - val_accuracy: 0.7111 - val_loss: 0.0000e+00
Epoch 9/15
6/6 - 0s - 11ms/step - accuracy: 0.6875 - loss: 0.0000e+00 - 

In [23]:
embedding_dim = batch_size # 50 #batch_size
#maxlen = 15

model = Sequential()
model.add(Embedding(input_dim = max_features, 
                           output_dim = embedding_dim, 
                           input_length = maxlen))
model.add(Flatten())
model.add(Dense(15, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

history = model.fit(X_train_cvectorized, y_train_hsA,
                    epochs=15,
                    verbose=2,
                    validation_data=(X_test_cvectorized, y_test_hsA),
                    batch_size=batch_size)
                    
loss, accuracy = model.evaluate(X_train_cvectorized, y_train_hsA, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test_cvectorized, y_test_hsA, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
#plot_history(history)

# make predictions
testPredict_stackRNN = model.predict(X_test_cvectorized)
print('\t', 'Accuracy', accuracy_score(y_test_hsA, testPredict_stackRNN.round()))



c:\Users\histe\Documents\formacion\data_science_IA\proyecto_twitter_tech_sentiment_analysis\.venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/15
7/7 - 1s - 139ms/step - accuracy: 0.2036 - loss: 0.9221 - val_accuracy: 0.2000 - val_loss: 0.9013
Epoch 2/15
7/7 - 0s - 13ms/step - accuracy: 0.2036 - loss: 0.8584 - val_accuracy: 0.2000 - val_loss: 0.8890
Epoch 3/15
7/7 - 0s - 13ms/step - accuracy: 0.2217 - loss: 0.8453 - val_accuracy: 0.2105 - val_loss: 0.8767
Epoch 4/15
7/7 - 0s - 13ms/step - accuracy: 0.2217 - loss: 0.8382 - val_accuracy: 0.2105 - val_loss: 0.8646
Epoch 5/15
7/7 - 0s - 14ms/step - accuracy: 0.2217 - loss: 0.8310 - val_accuracy: 0.2105 - val_loss: 0.8614
Epoch 6/15
7/7 - 0s - 12ms/step - accuracy: 0.2217 - loss: 0.8289 - val_accuracy: 0.2105 - val_loss: 0.8596
Epoch 7/15
7/7 - 0s - 13ms/step - accuracy: 0.2217 - loss: 0.8280 - val_accuracy: 0.2105 - val_loss: 0.8579
Epoch 8/15
7/7 - 0s - 12ms/step - accuracy: 0.2217 - loss: 0.8230 - val_accuracy: 0.2105 - val_loss: 0.8572
Epoch 9/15
7/7 - 0s - 12ms/step - accuracy: 0.2217 - loss: 0.8218 - val_accuracy: 0.2105 - val_loss: 0.8561
Epoch 10/15
7/7 - 0s - 12ms

In [ ]:
### Embeding

In [24]:

lstm_out = 98

model = Sequential()
model.add(Embedding(input_dim = max_features, output_dim = embedding_dim, input_length = maxlen))
#model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

batch_size = 32
model.fit(X_train_cvectorized, y_train_hsA, epochs = 10, batch_size=batch_size, verbose = 2)

loss, accuracy = model.evaluate(X_train_cvectorized, y_train_hsA, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test_cvectorized, y_test_hsA, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
#plot_history(history)

# make predictions
testPredict_stackRNN = model.predict(X_test_cvectorized)
print('\t', 'Accuracy', accuracy_score(y_test_hsA, testPredict_stackRNN.round()))

c:\Users\histe\Documents\formacion\data_science_IA\proyecto_twitter_tech_sentiment_analysis\.venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
7/7 - 2s - 298ms/step - accuracy: 0.2715 - loss: 0.9748
Epoch 2/10
7/7 - 0s - 23ms/step - accuracy: 0.2036 - loss: 0.9053
Epoch 3/10
7/7 - 0s - 21ms/step - accuracy: 0.2036 - loss: 0.8858
Epoch 4/10
7/7 - 0s - 23ms/step - accuracy: 0.2036 - loss: 0.8767
Epoch 5/10
7/7 - 0s - 23ms/step - accuracy: 0.2081 - loss: 0.8661
Epoch 6/10
7/7 - 0s - 23ms/step - accuracy: 0.2217 - loss: 0.8633
Epoch 7/10
7/7 - 0s - 23ms/step - accuracy: 0.2217 - loss: 0.8512
Epoch 8/10
7/7 - 0s - 23ms/step - accuracy: 0.2217 - loss: 0.8606
Epoch 9/10
7/7 - 0s - 22ms/step - accuracy: 0.2217 - loss: 0.8556
Epoch 10/10
7/7 - 0s - 21ms/step - accuracy: 0.2217 - loss: 0.8770
Training Accuracy: 0.2036
Testing Accuracy:  0.2000
1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/stepWARNING:tensorflow:6 out of the last 9 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x000002AE29EE7560> triggered tf.function retracing. Tracing is expensive and the excessive number of tr

In [13]:
model.save('best_keras_network_model.h5')

In [11]:
input_list = list(map(lambda x: str(x), y_train_hsA.round()))
c = Counter( input_list )

print( c.items() )

dict_items([('[0 0 0]', 105), ('[0 1 0]', 41), ('[1 1 0]', 45), ('[0 0 1]', 14), ('[1 0 1]', 1)])


In [20]:
corpus_dev_esA["predict_test"] = list(map(lambda x: str(x), testPredict_stackRNN.round()))

corpus_dev_esA.to_csv("tweets_search_etiquetas_clean_test_predict.csv", sep="\t", encoding = "latin-1")

In [21]:
list(map(lambda x: str(x), testPredict_stackRNN.round()))

['[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 0. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0.]',
 '[0. 1. 0

In [7]:
from dash import Dash, html, dcc
import plotly.express as px
from dash.dependencies import Input, Output
import pandas as pd

app = Dash(__name__)

# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options
"""
df = pd.DataFrame({
    "Ambito": ["funcionamiento", "atencion_cliente", "atencion_cliente", "funcionamiento", "funcionamiento", "funcionamiento"],
    "Conteo": [4, 1, 2, 2, 4, 5],
    "Sentimiento": ["Positivo", "Negativo", "Negativo", "Negativo", "Negativo", "Positivo"]
})
"""


app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Analisis de sentimiento
    '''),

    dcc.Graph(
        id='live-update-graph'#,
        #figure=fig
    ),
    dcc.Interval(
            id='interval-component',
            interval=8*1000, # in milliseconds
            n_intervals=0
    )
])


# Multiple components can update everytime interval gets fired.
@app.callback(Output('live-update-graph', 'figure'),
              Input('interval-component', 'n_intervals'))
def update_graph_live(n):

    df = pd.read_csv("stream_tweet_groupby.csv", sep = ",", encoding= "latin-1", dtype = {'sentiment_code': str})

    df["sentiment_bin"] = df["sentiment_code"].apply(lambda x: str(x[0]))
    sentiment_dict = {"0": "negativo", "1": "positivo"}
    df["sentiment_bin"] = df["sentiment_bin"].apply(lambda x: sentiment_dict[x])

    fig = px.bar(df, x="sentiment_code", y="count", color ="sentiment_bin", barmode="group")
    #fig = px.bar(df, x="Fruit", y="Amount", color="City", barmode="group")

    return fig


if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [1]:

from search_or_simulate import SearchOrSimulate
from api_twitter import StreammerTwitter


streammer = SearchOrSimulate()
#streammer.simulate_search_streaming("tweets_search_2.txt", True, model_file = "best_keras_network_model.h5", cvectorizer_file = "count_vectorizer.joblib")
#streammer.search_streaming("streaming_tweets_search.txt", False, model_file = "best_keras_network_model.h5", cvectorizer_file = "count_vectorizer.joblib")
streammer.search_30_days_ago()
#transformed_tweet



ModuleNotFoundError: No module named 'search_or_simulate'